In [2]:
from langchain_community.document_loaders import PyPDFLoader

loader=PyPDFLoader("pdfs/2410.09836v2.pdf")
pages=loader.load_and_split()

In [3]:
pages[0]

Document(metadata={'source': 'pdfs/2410.09836v2.pdf', 'page': 0}, page_content='Learning Pattern-Specific Experts for Time Series\nForecasting Under Patch-level Distribution Shift\nYanru Sun, Zongxia Xie∗, Emadeldeen Eldele‡†, Dongyue Chen, Qinghua Hu, Min Wu‡\nCollege of Intelligence and Computing, Tianjin University, China\n‡I2R, Agency for Science, Technology and Research, Singapore.\n†Department of Computer Science, Khalifa University, UAE.\n{yanrusun, dyuechen, huqinghua}@tju.edu.cn,caddiexie@hotmail.com,\nemad0002@ntu.edu.sg,wumin@i2r.a-star.edu.sg\nAbstract\nTime series forecasting, which aims to predict future values based on historical\ndata, has garnered significant attention due to its broad range of applications.\nHowever, real-world time series often exhibit heterogeneous pattern evolution\nacross segments, such as seasonal variations, regime changes, or contextual shifts,\nmaking accurate forecasting challenging. Existing approaches, which typically\ntrain a single model 

In [4]:
from langchain_openai import ChatOpenAI,OpenAI

model=ChatOpenAI(
  model="qwen2.5-coder-1.5b-instruct",
  openai_api_key="EMPTY",
  base_url="http://127.0.0.1:1234/v1",
  temperature=0.3
)

In [5]:
docs=""
for item in pages:
    docs+=item.page_content
  
docs

'Learning Pattern-Specific Experts for Time Series\nForecasting Under Patch-level Distribution Shift\nYanru Sun, Zongxia Xie∗, Emadeldeen Eldele‡†, Dongyue Chen, Qinghua Hu, Min Wu‡\nCollege of Intelligence and Computing, Tianjin University, China\n‡I2R, Agency for Science, Technology and Research, Singapore.\n†Department of Computer Science, Khalifa University, UAE.\n{yanrusun, dyuechen, huqinghua}@tju.edu.cn,caddiexie@hotmail.com,\nemad0002@ntu.edu.sg,wumin@i2r.a-star.edu.sg\nAbstract\nTime series forecasting, which aims to predict future values based on historical\ndata, has garnered significant attention due to its broad range of applications.\nHowever, real-world time series often exhibit heterogeneous pattern evolution\nacross segments, such as seasonal variations, regime changes, or contextual shifts,\nmaking accurate forecasting challenging. Existing approaches, which typically\ntrain a single model to capture all these diverse patterns, often struggle with the\npattern drifts 

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

template="""
'''
{context}
'''
总结上面的内容
"""

prompt=ChatPromptTemplate.from_template(template)

output_parser=StrOutputParser()

In [12]:
chain=prompt | model | output_parser
for chunk in chain.stream({"context":docs[:2000]}):
    print(chunk,end="")

文章标题：基于模式特定专家的时序预测方法：适应分段分布变化的时间序列 forecasting

作者：Yanru Sun, Zongxia Xie, Emadeldeen Eldele, Dongyue Chen, Qinghua Hu, Min Wu

摘要：
时间序列预测，旨在根据历史数据预测未来的值，因其广泛的应用而受到重视。然而，现实世界的时间序列往往表现出不同段的模式演变，如季节变化、 regime切换或上下文变化，这使得准确预测变得困难。现有方法，通常将单个模型训练为捕捉所有这些不同的模式，经常在分段间出现模式 drift，并可能导致泛化不佳。为此，我们提出TFPS（Time Series Pattern-Specific Experts），一种基于模式特定专家的时序预测方法。TFPS通过使用双域编码器来捕获时间域和频率域特征，从而更全面地理解时间动态。然后，它通过动态识别数据段中的不同模式进行子空间聚类。最后，这些模式被建模为专门的专家，允许模型学习多个预测函数。在实验证明中，TFPS在表现出显著分布变化的时间序列数据集上优于现有方法，特别是在这种情况下。数据和代码可用：https://github.com/syrGitHub/TFPS。

In [13]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

loader=PyPDFDirectoryLoader("pdfs")

docs=loader.load()
docs

[Document(metadata={'source': 'pdfs\\2410.09836v2.pdf', 'page': 0}, page_content='Learning Pattern-Specific Experts for Time Series\nForecasting Under Patch-level Distribution Shift\nYanru Sun, Zongxia Xie∗, Emadeldeen Eldele‡†, Dongyue Chen, Qinghua Hu, Min Wu‡\nCollege of Intelligence and Computing, Tianjin University, China\n‡I2R, Agency for Science, Technology and Research, Singapore.\n†Department of Computer Science, Khalifa University, UAE.\n{yanrusun, dyuechen, huqinghua}@tju.edu.cn,caddiexie@hotmail.com,\nemad0002@ntu.edu.sg,wumin@i2r.a-star.edu.sg\nAbstract\nTime series forecasting, which aims to predict future values based on historical\ndata, has garnered significant attention due to its broad range of applications.\nHowever, real-world time series often exhibit heterogeneous pattern evolution\nacross segments, such as seasonal variations, regime changes, or contextual shifts,\nmaking accurate forecasting challenging. Existing approaches, which typically\ntrain a single mode

In [16]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

embeddings=OpenAIEmbeddings(
  base_url="http://127.0.0.1:1234/v1",
  openai_api_key="lm-studio",
  model="text-embedding-bge-large-zh-v1.5",
  check_embedding_ctx_length=False
)

In [17]:
vectorstoreDB=FAISS.from_documents(docs,embeddings)
vectorstoreDB

In [18]:
retriver=vectorstoreDB.as_retriever(
     search_type="similarity_score_threshold",
     search_kwargs={"score_threshold": 0.3}
)

In [19]:
from langchain_core.prompts import ChatPromptTemplate

template="""
只根据以下文档回答问题：
{context}

问题：{question}
"""

prompt=ChatPromptTemplate.from_template(template)

In [24]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough,RunnableParallel

chain=(
    {"context": retriver, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

for chunk in chain.stream("时间序列预测是什么"):
    print(chunk,end="")

时间序列预测是一种用于预测未来数据的方法，它基于历史数据来预测未来的值。